In [ ]:
import pandas as pd
import numpy as np
import data_handler as dh
import model as md
import time
import matplotlib.pyplot as plt
from matplotlib import rc
from multiprocessing import Pool
import pickle
pd.options.mode.chained_assignment = None 
import copy
from scipy.stats import norm
import random


In [ ]:
import importlib
importlib.reload(dh)
importlib.reload(md)

In [ ]:
import numba
numba.__version__

In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
from platform import python_version
print('python', python_version())
print('numpy', np.__version__)
print('pandas', pd.__version__)
import graph_tool
print('graph_tool', graph_tool.__version__)

# Beckmann model

parameter $\mu = 0.25$

In [ ]:
beckmann_save = 'beckmann_results/'

In [ ]:
net_name = 'Anaheim_net.tntp'
trips_name = 'Anaheim_trips.tntp'

handler = dh.DataHandler()
graph_data = handler.GetGraphData(net_name, columns = ['init_node', 'term_node', 'capacity', 'free_flow_time'])
graph_correspondences, total_od_flow = handler.GetGraphCorrespondences(trips_name)

model = md.Model(graph_data, graph_correspondences, 
                    total_od_flow, mu = 0.25, rho = 0.15)

graph_data['graph_table'].head()

In [ ]:
epsilons = np.logspace(3,-0.5,8)
epsilons = [epsilons[4]]

In [ ]:
assert(model.mu == 0.25)
max_iter = 10000
for index, eps_abs in enumerate(epsilons):
    if index < len(epsilons) - 1:
        continue
    print('eps_abs =', eps_abs)
    solver_kwargs = {'eps_abs': eps_abs,
                     'max_iter': max_iter, 'stop_crit': 'dual_gap',
                     'verbose' : True, 'verbose_step': 4000, 'save_history' : True}
    tic = time.time()
    result = model.find_equilibrium(solver_name = 'ustm', composite = True, solver_kwargs = solver_kwargs)
    toc = time.time()
    print('Elapsed time: {:.0f} sec'.format(toc - tic))
    print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
    print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

    result['eps_abs'] = eps_abs
    result['elapsed_time'] = toc - tic
    with open(beckmann_save + 'anaheim_result_' + 'ustm' + '_eps_abs_' + str(index) + '_beckmann.pickle', 'wb') as f:
        pickle.dump(result, f)

In [ ]:
old_result = result.copy()

In [ ]:
def node_roads(graph_table, node):
    return set(graph_table[np.array(graph_table['term_node'] == node) + np.array(graph_table['init_node'] == node)].index)


def edge_neighbours(graph_table, edge_num):
    init_node = graph_table.loc[edge_num]['init_node']
    term_node = graph_table.loc[edge_num]['term_node']
    res = node_roads(graph_table, init_node)
    res.update(node_roads(graph_table, term_node))
    return res

def n_neighbour_edges(graph_table, node, depth):
    neighbours = node_roads(graph_table, node)
    for i in range(depth - 1):
        new_neighbours = neighbours.copy()
        for curr_edge in neighbours:
            new_neighbours.update(edge_neighbours(graph_table, curr_edge))
        neighbours = new_neighbours
    return neighbours


def normal_distribution_noise(graph_data, graph_correspondences, node, depth, std_percentage_capacity, std_percentage_flow):
    graph_data_copy = copy.deepcopy(graph_data)
    graph_correspondences_copy = copy.deepcopy(graph_correspondences)
    graph_capacity = graph_data_copy['graph_table']['capacity']
    graph_free_flow_time = graph_data_copy['graph_table']['free_flow_time']

    graph_table = graph_data['graph_table']
    neighbour_edges = n_neighbour_edges(graph_table, node, depth)

    for index in neighbour_edges:
        capacity = graph_capacity[index]
        std = capacity * std_percentage_capacity
        noise = norm.rvs(0, std)
        graph_capacity[index] = capacity + noise

        flow = graph_free_flow_time[index]
        std = flow * std_percentage_flow
        noise = norm.rvs(0, std)
        graph_free_flow_time[index] = flow + noise
    return graph_data_copy, graph_correspondences_copy

In [ ]:
std_percentage_capacity = 0.05
std_percentage_flow = 0.05
node = random.randint(1, graph_data['nodes number'])
print(node)
depth = 2
new_graph_data, new_graph_correspondences = normal_distribution_noise(graph_data, graph_correspondences, node, depth, std_percentage_capacity, std_percentage_flow)

In [ ]:
new_model = md.Model(new_graph_data, new_graph_correspondences, 
                    total_od_flow, mu = 0.25, rho = 0.15)

In [ ]:
assert(model.mu == 0.25)
max_iter = 40000
for index, eps_abs in enumerate(epsilons):
    if index < len(epsilons) - 1:
        continue
    print('eps_abs =', eps_abs)
    solver_kwargs = {'eps_abs': eps_abs,
                     'max_iter': max_iter, 'stop_crit': 'dual_gap',
                     'verbose' : True, 'verbose_step': 4000, 'save_history' : True}
    tic = time.time()
    result = new_model.find_equilibrium(t_start = old_result['times'],solver_name = 'ustm', composite = True, solver_kwargs = solver_kwargs)
    toc = time.time()
    print('Elapsed time: {:.0f} sec'.format(toc - tic))
    print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
    print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

    result['eps_abs'] = eps_abs
    result['elapsed_time'] = toc - tic
    with open(beckmann_save + 'anaheim_result_' + 'ustm' + '_eps_abs_' + str(index) + '_beckmann.pickle', 'wb') as f:
        pickle.dump(result, f)